# Examples for running FLORIS

In [1]:
# load modules
from floris.floris import Floris
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Setup floris and process input file

In [2]:
floris = Floris("example_input.json")

## Calculate Wake

In [3]:
import time

t1 = time.time()

floris.farm.flow_field.calculate_wake()

t2 = time.time()

print('Time to compute wake = ', t2-t1, 's')

Time to compute wake =  0.012473821640014648 s


## Compute Velocities at each Turbine

In [28]:
# gather farm items from FLORIS including turbines
farm_items = floris.farm.turbine_map.items

print("\tx\ty\tz\t  Cp\t Ct\t power\t\t a\t\t wind speed\t TI")
for coord, turbine in farm_items:
    print("{} | {:<7.3f} {:<7.3f} {:<15.3f} {:<15.3f} {:<15.3f} {:<15.3f}".format(coord, turbine.Cp, turbine.Ct, turbine.power, turbine.aI, turbine.average_velocity, turbine.turbulence_intensity))

	x	y	z	  Cp	 Ct	 power		 a		 wind speed	 TI
<   0.000,    0.000,   90.000> | 0.463   0.766   1712005.168     0.258           7.851           0.060          
< 800.000,    0.000,   90.000> | 0.459   0.813   946340.408      0.284           6.462           0.127          
<   0.000,  630.000,   90.000> | 0.463   0.766   1712005.168     0.258           7.851           0.060          
< 800.000,  630.000,   90.000> | 0.459   0.813   946340.408      0.284           6.462           0.127          


## Optimize with wake steering

In [30]:
import floris.optimization as flopt

# Run floris with no yaw
floris.farm.set_yaw_angles(0.0, calculate_wake=True)

# Determine initial power production
power_initial = np.sum([turbine.power for turbine in floris.farm.turbines])

# Set bounds for the optimization on the yaw angles (deg)
minimum_yaw, maximum_yaw = 0.0, 25.0

# Compute the optimal yaw angles
opt_yaw_angles = flopt.wake_steering(floris, minimum_yaw, maximum_yaw)
print('Optimal yaw angles for:')
for i, yaw in enumerate(opt_yaw_angles):
    print('Turbine ', i, ' yaw angle = ', np.degrees(yaw))

# Calculate power gain with new yaw angles
floris.farm.set_yaw_angles(opt_yaw_angles, calculate_wake=True)

# Optimal power
power_optimal = np.sum([turbine.power for turbine in floris.farm.turbines])
print('Power increased by {}%'.format(100 * (power_optimal - power_initial) / power_initial))

Optimal yaw angles for:
Turbine  0  yaw angle =  24.999999987589906
Turbine  1  yaw angle =  8.152804870526513e-09
Turbine  2  yaw angle =  24.999999987553267
Turbine  3  yaw angle =  8.133263542563601e-09
Power increased by 6.222567052402905%
